In [1]:
from pyspark.sql import functions as sparkf

raw_df = spark.createDataFrame(
 [
  ('Store 1',1,448),
  ('Store 1',2,None),
  ('Store 1',3,499),
  ('Store 1',44,432),
  (None,None,None),
  ('Store 2',1,355),
  ('Store 2',1,355),
  ('Store 2',None,345),
  ('Store 2',3,387),
  ('Store 2',4,312)
],
 ['Store','WeekInMonth','Revenue']
)


In [2]:
raw_df.printSchema()

root
 |-- Store: string (nullable = true)
 |-- WeekInMonth: long (nullable = true)
 |-- Revenue: long (nullable = true)



In [3]:
raw_df.count()

raw_df.show()

+-------+-----------+-------+
|  Store|WeekInMonth|Revenue|
+-------+-----------+-------+
|Store 1|          1|    448|
|Store 1|          2|   null|
|Store 1|          3|    499|
|Store 1|         44|    432|
|   null|       null|   null|
|Store 2|          1|    355|
|Store 2|          1|    355|
|Store 2|       null|    345|
|Store 2|          3|    387|
|Store 2|          4|    312|
+-------+-----------+-------+



In [4]:
#นำ Bucketizer ที่ใช้ convert Numeric เป็น Categorical มาใช้

from pyspark.ml.feature import Bucketizer


In [5]:
#คำนวณค่า Mean, Stddev จาก “WeekInMonth” ของ “Store 1”


mean_stddev = raw_df.filter(sparkf.col('Store') == 'Store 1')\
.groupBy('Store')\
.agg(sparkf.mean('WeekInMonth').alias('M')\
     ,sparkf.stddev('WeekInMonth').alias('SD'))\
.select(['M','SD'])\
.collect()[0]


In [6]:
mean_stddev

Row(M=12.5, SD=21.01586702153082)

In [7]:
#ทดลองดึงค่าออกมาแสดง

mean_val = mean_stddev['M']

stddev_val = mean_stddev['SD']


In [8]:
mean_val

12.5

In [9]:
stddev_val

21.01586702153082

In [10]:
#คำนวนค่าอื่นๆ

mini = max((mean_val-stddev_val),0)

maxi = mean_val + stddev_val


In [11]:
mini


0

In [12]:
maxi


33.51586702153082

In [13]:
#สร้าง transformer สำหรับ Binning Method ที่มีการกำหนดช่วงค่าสำหรับแบ่งแยกข้อมูล

b = Bucketizer(splits=[mini, mean_val, maxi, float('inf')],\
               inputCol='WeekInMonth',outputCol='Bin')


In [14]:
#นำ transformer ไปใช้แบ่งแยกข้อมูลสำหรับประเมิน Outliers

b.transform(raw_df.select('WeekInMonth')\
            .filter(sparkf.col('Store') == 'Store 1')).show()


+-----------+---+
|WeekInMonth|Bin|
+-----------+---+
|          1|0.0|
|          2|0.0|
|          3|0.0|
|         44|2.0|
+-----------+---+



In [15]:
mean_val + (2*(stddev_val))

54.53173404306164

In [16]:
#Assign ผลลัพธ์จากการแบ่งแยกค่า “WeekInMonth” ให้กับตัวแปรใหม่

bucketizedWeekInMonth_df = b.transform(raw_df.select('WeekInMonth')\
                                       .filter(sparkf.col('Store') == 'Store 1'))


In [17]:

#ทำแจงนับความถี่ในแต่ละ Bucket/Bin เพื่อประเมินจำนวน Outliers

bucketizedWeekInMonth_df.groupBy('Bin').count().show()


+---+-----+
|Bin|count|
+---+-----+
|0.0|    3|
|2.0|    1|
+---+-----+



#### ระบุ Extreme Value ด้วย IQR และ Upper/Lower Fence

In [18]:
onlyStore1_df = raw_df.filter(sparkf.col('Store') == 'Store 1')\
.select('WeekInMonth').withColumnRenamed('WeekInMonth','value')

In [19]:
onlyStore1_df.show()

+-----+
|value|
+-----+
|    1|
|    2|
|    3|
|   44|
+-----+



In [20]:
from pyspark.sql.types import *
#from pyspark.sql import functions as sparkf

In [21]:
import pyspark.sql.functions as f

In [22]:
def calculate_bounds(df):
    bounds = {
        c: dict(
            zip(["q1", "q3"], df.approxQuantile(c, [0.25, 0.75], 0))
        )
        for c,d in zip(df.columns, df.dtypes) if ((d[1] == "int")|(d[1] == "bigint"))
    }
    #print (bounds)
    for c in bounds:
        iqr = bounds[c]['q3'] - bounds[c]['q1']
        #print (iqr)
        bounds[c]['min'] = bounds[c]['q1'] - (iqr * 1.5)
        bounds[c]['max'] = bounds[c]['q3'] + (iqr * 1.5)

    return bounds

In [24]:
raw_df.filter(sparkf.col('Store') == 'Store 1')\
.filter(sparkf.col('WeekInMonth') > calculate_bounds(onlyStore1_df)['value']['max']).show()

+-------+-----------+-------+
|  Store|WeekInMonth|Revenue|
+-------+-----------+-------+
|Store 1|         44|    432|
+-------+-----------+-------+

